In [1]:
from fastai.vision import *

In [2]:
import pandas as pd

In [3]:
TEST_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/test/")

In [4]:
! ls $TEST_PATH

images	metadata.json


In [5]:
import json

In [6]:
with open(TEST_PATH/"metadata.json", encoding="utf8", errors='ignore') as json_file:
    data = json.load(json_file)

In [7]:
data.keys()

dict_keys(['images', 'info', 'licenses'])

In [8]:
df =  pd.DataFrame.from_dict(data["images"])

In [12]:
df.iloc[0]["file_name"]

'images/104/104891.jpg'

In [11]:
df.head()

,file_name,height,id,license,width
0,images/104/104891.jpg,1000,104891,1,661
1,images/018/18029.jpg,1000,18029,1,661
2,images/035/35151.jpg,1000,35151,1,661
3,images/124/124144.jpg,1000,124144,1,682
4,images/024/24649.jpg,1000,24649,1,682


In [12]:
## Submission

In [ ]:
from fastai.vision import ImageImageList
test_data = ImageImageList.from_df(df_test, PATH/'test_images', cols="file_name")
learn.data.add_test(test_data)
preds, y = learn.get_preds(ds_type=DatasetType.Test)